In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import seaborn as sns
from sklearn.ensemble import IsolationForest
import joblib
os.chdir('Resources/')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('10_SP_Preprocessed_Data.csv')

X = df.drop(['HeartDisease'], axis='columns')
Y = df[['HeartDisease']]

In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
import joblib

mx = 0
ind = 0
seed = 0

for i in range(5000):
    seed = i
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    f1_scores = []

    for fold, (train_index, test_index) in enumerate(skf.split(X, Y)):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

        X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        model = Sequential()
        model.add(LSTM(11, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
        model.add(Dropout(0.01 * 5))
        model.add(Dense(1, activation='sigmoid'))

        optimizer = Adam(learning_rate=1/15)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        model.fit(X_train_reshaped, Y_train, epochs=1, batch_size=10000, verbose=0)

        X_train_features = model.predict(X_train_reshaped)
        X_test_features = model.predict(X_test_reshaped)

        rf_lstm = RandomForestClassifier(n_estimators=47, random_state=seed)
        rf_lstm.fit(X_train_features, Y_train.values.ravel())

        Y_pred = rf_lstm.predict(X_test_features)
        performance_lstm = f1_score(Y_test, Y_pred)

        f1_scores.append(performance_lstm)

        X_test_with_index = X_test.copy()
        X_test_with_index['SL'] = X.index[test_index]

        avg_f1 = np.mean(f1_scores)
    
    if (avg_f1>mx):
        mx=avg_f1
        ind=i

    print(i, avg_f1, mx, ind)

114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33 0.8469338152156606 0.8553678333144363 6
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━